In [ ]:
import os
import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset
from makitorch.dataloaders.HsiDataloader import HsiDataloader

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json

# Script with experiment code
from my_super_net_parts_diff_window_size_opt_change_lr_dyn_new_augs import *
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [ ]:
cut_window=(128, 128); image_shape=(512, 512); num_classes=17; divided_batch=4; 

In [ ]:
net = MySuperNetLittleInput(17, 17)
net.eval()
with torch.no_grad():
    _ = net(torch.randn(1, 17, cut_window[0], cut_window[1]))
net.train()

In [ ]:
import glob
w_sorted = sorted(glob.glob('pytorch_li_logs/rerun=1/(run=1)Erase_test_cut_window=(128, 128)_change_lr_dyn_arch_200ep_PCA_RustamPreprocess(k=1)_CE_cosine(t_0=2,t_mul=1)_arch_type=MySuperNetLittleInput_batch_size=128_lr=0.001/*'), 
       key=lambda x: -float(x.split('/')[-1].split('-')[-1][9:13])
)

pick_best_one = w_sorted[0]
w_sorted

In [ ]:
model = NnModel.load_from_checkpoint(
    pick_best_one,
    loss=None, model=net
)

In [ ]:
device = 'cuda:0'

net = model.model
net.to(device=device)
net.eval()

In [ ]:
test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')
# Test
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=test_indices, cut_window=None,
    create_shared_memory=False
)
dataset_test = HsiDataloaderCutter(
    images=dataset_creator_test.images, masks=dataset_creator_test.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=None,
)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
all_preds_list = []
all_target_list = []

for in_data_x, val_data in tqdm(test_loader): # max test size == 39
    batch_size = in_data_x.shape[0]
    # We will cut image into peases and stack it into single BIG batch
    h_win, w_win = cut_window
    h_parts, w_parts = image_shape[1] // w_win, image_shape[0] // h_win
    in_data_x_parts_list = cut_into_parts_model_input(
        in_data_x, h_parts=h_parts, 
        w_parts=w_parts, h_win=h_win, w_win=w_win
    )
    in_data_x_batch = torch.cat(in_data_x_parts_list, dim=0) # (N, 17, 1, 1)
    # Make predictions via batch in order to 
    # speed up calculation and do not eat all memory
    part_divided = len(in_data_x_batch) // divided_batch
    pred_batch_list = []

    for b_i in range(part_divided):
        if b_i == (part_divided - 1):
            # last
            single_batch = in_data_x_batch[b_i * divided_batch:]
        else:
            single_batch = in_data_x_batch[b_i * divided_batch: (b_i+1) * divided_batch]
        # Make predictions
        with torch.no_grad():
            data = single_batch.to(device=device)
            preds = net(data).cpu().detach() # (divided_batch, C, H, W)
        pred_batch_list.append(preds)
    pred_batch_tensor = torch.cat(pred_batch_list, dim=0)
    pred_mask = merge_parts_into_single_mask(
        preds=pred_batch_tensor, shape=(batch_size, num_classes, image_shape[0], image_shape[1]), # (N, 17, 512, 512) 
        h_parts=h_parts, w_parts=w_parts, h_win=h_win, w_win=w_win
    )
    all_preds_list.append(pred_mask) 
    all_target_list.append(val_data)

all_preds_mask_list = [
    torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    for preds in all_preds_list
]


all_preds_tensor = torch.cat(all_preds_list, dim=0)
all_target_tensor = torch.cat(all_target_list, dim=0).squeeze()
all_preds_mask_tensor = torch.cat(all_preds_mask_list, dim=0)

In [ ]:
all_preds_tensor.shape, all_target_tensor.shape, all_preds_mask_tensor.shape

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
sns.heatmap(all_preds_mask_tensor[0], vmin=0, vmax=17)

fig.add_subplot(1, 2, 2)
sns.heatmap(all_target_tensor[0], vmin=0, vmax=17)


In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
sns.heatmap(all_preds_mask_tensor[1], vmin=0, vmax=17)

fig.add_subplot(1, 2, 2)
sns.heatmap(all_target_tensor[1], vmin=0, vmax=17)


In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
sns.heatmap(all_preds_mask_tensor[2], vmin=0, vmax=17)

fig.add_subplot(1, 2, 2)
sns.heatmap(all_target_tensor[2], vmin=0, vmax=17)


## Calculate metrics

In [ ]:
def create_big_image(tensor, single_elem_shape):
    shape = tensor.shape
    tensor = torch.cat([
        t_s.view(*single_elem_shape)
        for t_s in tensor
    ], dim=-1)
    return tensor

pred_big_tensor = create_big_image(torch.clone(all_preds_tensor), single_elem_shape=[1, 17, 512, 512]) # Final shape!
pred_mask_big_tensor = create_big_image(torch.clone(all_preds_mask_tensor), single_elem_shape=[1, 512, 512]) # Final shape!
target_big_tensor = create_big_image(torch.clone(all_target_tensor), single_elem_shape=[1, 512, 512])  # Final shape!

target_one_hotted_tensor = torch.nn.functional.one_hot(
    target_big_tensor, 17 # Num classes
)
# (N, H, W, C) --> (N, C, H, W)
target_one_hotted_tensor = target_one_hotted_tensor.permute(0, -1, 1, 2)
dice_loss_val = dice_loss(
    pred_big_tensor, target_one_hotted_tensor, 
    dim=[0, 2, 3], use_softmax=True, softmax_dim=1,
)
metric = calculate_iou(pred_big_tensor, target_big_tensor)

if isinstance(metric, tuple):
    metric = metric[0]


# Show metrics results
mean_dice = dice_loss_val.mean().float()
mean_dice_per_class = dice_loss_val.float()
mean_iou = metric.mean()
mean_iou_per_class = metric.mean(axis=0)


print(f"Mean dice = {mean_dice}")
print("Mean dice per class:")
for i in range(17):
    print(f"| class={i} dice={mean_dice_per_class[i]}")

print(f"Mean iou = {mean_iou}")
print("Mean iou per class:")
for i in range(17):
    print(f"| class={i} iou={mean_iou_per_class[i]}")


## f1 

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(
    np.asarray(target_big_tensor).reshape(-1),
    np.asarray(pred_mask_big_tensor).reshape(-1), 
    average='macro'
)

In [ ]:
f1_score(
    np.asarray(target_big_tensor).reshape(-1),
    np.asarray(pred_mask_big_tensor).reshape(-1), 
    average='weighted'
)

In [ ]:
f1_score(
    np.asarray(target_big_tensor).reshape(-1),
    np.asarray(pred_mask_big_tensor).reshape(-1), 
    average=None
).mean()